In [0]:
dbutils.widgets.text("catalog_name","")
dbutils.widgets.text("schema_name","")
catalog_name=dbutils.widgets.get("catalog_name")
schema_name=dbutils.widgets.get("schema_name")

In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read.table(f"{catalog_name}.{schema_name}.medication_data")
bronze_df = df.withColumn("test_date", to_date(col("test_date"))).drop("_rescued_data")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
from delta.tables import DeltaTable



# Read the existing Silver Table (Delta Table)
try:
    silver_table = DeltaTable.forName(spark, f"{catalog_name}.silver.medication_data")
except:
    silver_table = None

# Define window to keep latest record per `patient_id`
window_spec = Window.partitionBy("patient_id").orderBy(col("source_file").desc())

# Deduplicate latest records
bronze_dedup_df = (
    bronze_df.withColumn("row_num", row_number().over(window_spec))
    .filter(col("row_num") == 1)
    .drop("row_num")
)

# Merge new data into Silver Table
if silver_table:
    silver_table.alias("silver").merge(
        bronze_dedup_df.alias("bronze"),
        "silver.patient_id = bronze.patient_id"
    ).whenMatchedUpdate(set={
        "test_name": col("bronze.test_name"),
        "result": col("bronze.result"),
        "normal_range": col("bronze.normal_range"),
        "test_date": col("bronze.test_date"),
        "lab_name": col("bronze.lab_name"),
        "source_file": col("bronze.source_file"),
    }).whenNotMatchedInsert(values={
        "patient_id": col("bronze.patient_id"),
        "test_name": col("bronze.test_name"),
        "result": col("bronze.result"),
        "normal_range": col("bronze.normal_range"),
        "test_date": col("bronze.test_date"),
        "lab_name": col("bronze.lab_name"),
       
        "source_file": col("bronze.source_file"),
    }).execute()
else:
    bronze_dedup_df.write.format("delta").saveAsTable(f"{catalog_name}.silver.medication_data")

print("Incremental load to Silver Layer completed successfully!")